# Polling data capture from Wikipedia

**Make sure to:**
 * run before doing any analysis; and
 * check the data validation before moving on to the analysis.

## Python setup

In [1]:
# analytic imports
import pandas as pd

In [2]:
# local imports
import data_capture as dc
from common import ATTITUDINAL, MIDDLE_DATE, VOTING_INTENTION

## Get raw polling data from Wikipedia

Note: web-scraping of data is fragile. 
This code will need to be checked from time to time.

### Get all tables from the Wikipedia web page

In [3]:
URL = (
    "https://en.wikipedia.org/wiki/" +
    "Opinion_polling_for_the_2025_Australian_federal_election"
    #"Opinion_polling_for_the_next_Australian_federal_election"  # changed November 2025
)
df_list = dc.get_table_list(URL)
print(f"Total number of tables on page: {len(df_list)}")

Total number of tables on page: 41


In [4]:
# KEEP CELL - quick overview of all the tables at Wiki ...
if False:  # True to print
    for i, table in enumerate(df_list):
        print(f"{i}: {table.columns}\n")

### Data select, merge and clean

Note: For this election cycle, Wikipedia has separate tables for each
calendar year. These tables will need to be updated below each year`m

In [5]:
# The Wikipedia table numbers will need updating each year ...
# And whenever the Wikipedia page is reorganised (such that the tables are renumbered)
VOTING_TABLES = (3, 4, 5, 6)
ATTITUDINAL_TABLES = (7, 8, 9)  

prep = {VOTING_INTENTION: VOTING_TABLES, ATTITUDINAL: ATTITUDINAL_TABLES}
data = {}
for label, table_list in prep.items():
    print("Collating:", label, table_list)
    table = dc.get_combined_table(df_list, table_list, verbose=False).copy()
    table = dc.clean(table)
    data[label] = table
    print(f"{label}: {len(table)} rows {table.index}")
    

Collating: voting-intention (3, 4, 5, 6)
voting-intention: 221 rows RangeIndex(start=0, stop=221, step=1)
Collating: attitudinal (7, 8, 9)
attitudinal: 125 rows RangeIndex(start=0, stop=125, step=1)


### Quick look at most recent N polls

In [6]:
# Let's look at the last N polls
N = 3
for label, table in data.items():
    print(f"{label}:")
    #display(table.head(N))
    display(table.tail(N))
    print()

voting-intention:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
218,11–15 December 2024,Essential,Online,1151.0,35.0,30.0,13.0,6.0,1.0,11.0,5.0,47.0,48.0,2024-12-11,2024-12-13,2024-12-15
219,13–15 December 2024,Freshwater Strategy,Online,1051.0,40.0,30.0,14.0,NaN,NaN,16.0,NaN,49.0,51.0,2024-12-13,2024-12-14,2024-12-15
220,30 December 2024–5 January 2025,Roy Morgan,Online,NaN,40.5,31.0,12.0,3.5,NaN,13.0,NaN,47.0,53.0,2024-12-30,2025-01-02,2025-01-05



attitudinal:


,Date,Firm,Interview mode,Sample,Preferred prime minister Albanese,Preferred prime minister Dutton,Preferred prime minister Don't Know,Preferred prime minister Net,Albanese Satisfied,Albanese Dissatisfied,Albanese Don't Know,Albanese Net,Dutton Satisfied,Dutton Dissatisfied,Dutton Don't Know,Dutton Net,First Date,Mean Date,Last Date
122,4–8 December 2024,Resolve Strategic,Online,NaN,35.0,35.0,30.0,0.0,31.0,57.0,12.0,-26.0,40.0,42.0,18.0,-2.0,2024-12-04,2024-12-06,2024-12-08
123,11–15 December 2024,Essential,Online,NaN,NaN,NaN,NaN,NaN,39.0,50.0,11.0,-11.0,44.0,41.0,15.0,3.0,2024-12-11,2024-12-13,2024-12-15
124,13–15 December 2024,Freshwater Strategy,Online,NaN,46.0,43.0,11.0,3.0,34.0,51.0,15.0,-17.0,37.0,40.0,23.0,-3.0,2024-12-13,2024-12-14,2024-12-15


### Standardise column names

In [7]:
fix = {
    # from : to
    "Firm": "Brand",
    "Sample": "Sample size",
}

for label, table in data.items():
    for old_col, new_col in fix.items():
        fix_me_list = table.columns[table.columns.str.contains(old_col, case=False)]
        if len(fix_me_list) == 1:
            fix_me_string = fix_me_list[0]
            table = table.rename(columns={fix_me_string: new_col})
            print(f"{label} fixed col from {fix_me_string} to {new_col}")
            data[label] = table

voting-intention fixed col from Sample size to Sample size
attitudinal fixed col from Firm to Brand
attitudinal fixed col from Sample to Sample size


### Remove MRP polls
MRP = multi-regression post-stratification polls

In [8]:
# Drop where interview mode is MRP
for label, table in data.items():
    drop_bool = (
        table['Brand'].str.contains('Accent Research', na=False) &
        table['Brand'].str.contains('RedBridge', na=False)
    )
    drop_index = drop_bool[drop_bool].index

    if len(drop_index) > 0:
        print(f"{label} MRP about to drop:")
        display(table.loc[drop_index])
        table = table.drop(drop_index)
        data[label] = table

voting-intention MRP about to drop:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
165,10 July – 27 August 2024,Accent Research/ RedBridge Group,Online,5976.0,38.0,32.0,12.0,NaN,NaN,18.0,NaN,50.0,50.0,2024-07-10,2024-08-03,2024-08-27
205,29 October – 20 November 2024,Accent Research/ RedBridge Group,Online,4909.0,39.0,31.0,11.0,NaN,NaN,19.0,NaN,49.0,51.0,2024-10-29,2024-11-09,2024-11-20


In [9]:
# Check for unusally large sample sizes - may be MRP polling
SAMPLE_CHECK = 3000

for label, table in data.items():
    sample_col = table.columns[table.columns.str.contains("sample", case=False)][0]
    odd = table.index[table[sample_col].notna() & (table[sample_col] >= SAMPLE_CHECK)]
    print(odd)
    if len(odd):
        print(f"{label}: --CHECK-- Based on sample size, these rows might be MRP data:")
        display(table.loc[odd])
        print('=' * 40)


Index([62, 172, 196], dtype='int64')
voting-intention: --CHECK-- Based on sample size, these rows might be MRP data:


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,Primary vote UND,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date
62,22 September – 4 October 2023,Resolve Strategic,Online,4728.0,31.0,37.0,12.0,7.0,2.0,11.0,NaN,57.0,43.0,2023-09-22,2023-09-28,2023-10-04
172,6–29 August 2024,Wolf & Smith,Online,10239.0,36.0,29.0,13.0,6.0,NaN,15.0,NaN,51.0,49.0,2024-08-06,2024-08-17,2024-08-29
196,14–25 October 2024,Australian National University,Online,3622.0,38.2,31.8,11.8,NaN,NaN,NaN,9.5,50.0,50.0,2024-10-14,2024-10-19,2024-10-25


Index([0, 38], dtype='int64')
attitudinal: --CHECK-- Based on sample size, these rows might be MRP data:


,Date,Brand,Interview mode,Sample size,Preferred prime minister Albanese,Preferred prime minister Dutton,Preferred prime minister Don't Know,Preferred prime minister Net,Albanese Satisfied,Albanese Dissatisfied,Albanese Don't Know,Albanese Net,Dutton Satisfied,Dutton Dissatisfied,Dutton Don't Know,Dutton Net,First Date,Mean Date,Last Date
0,23–31 May 2022,Morning Consult,Online,3770.0,NaN,NaN,NaN,NaN,51.0,24.0,25.0,27.0,NaN,NaN,NaN,NaN,2022-05-23,2022-05-27,2022-05-31
38,29 May – 12 June 2023,CT Group,Online,3000.0,NaN,NaN,NaN,NaN,42.0,36.0,22.0,6.0,NaN,NaN,NaN,NaN,2023-05-29,2023-06-05,2023-06-12


## Preliminary data validation

Note:Essential often does not distribute undecideds to the 2pp Vote share.

In [10]:
# Identify the groups of columns that should add across to 100
# We use this mechanism a few times below.

checkable_100: dict[str, list[str]] = {
    # label: [list of regex-patterns],
    VOTING_INTENTION: [
        r"Primary",
        r"2pp",
    ],
    ATTITUDINAL: [
        r"^Dutton (Satisfied|Dissatisfied|Don't Know)",
        r"^Albanese (Satisfied|Dissatisfied|Don't Know)",
        r"Preferred Prime Minister (Dutton|Albanese|Don't Know)",
    ],
}

In [11]:
# Check the columns that should add across to 100 actually do

if False:  # not always useful at this point
    for label, check_list in checkable_100.items():
        display(dc.row_sum_check(data[label], check_list))

## Distribute undecideds if the pollster has not

Mostly affects the Essential poll.

In [12]:
if dc.UNDECIDED_COLUMN in data[VOTING_INTENTION]:
    revised = dc.distribute_undecideds(
        table=data[VOTING_INTENTION].copy(),
        col_pattern_list=["Primary vote", "2pp vote"],
    )
    revised = revised.drop(columns=dc.UNDECIDED_COLUMN)
    data[VOTING_INTENTION] = revised
else:
    print("CHECK: this step was not applied")
    print("Most likely because it has already been applied.")

For Primary vote distributed undecideds over 23.74% of rows.
For 2pp vote distributed undecideds over 23.74% of rows.


## Forced data normalisation

Force columns that should sum to 100 to sum to 100.

This is an aggressive treatment, and the rows being forced into
submission need to be considered and reflected upon from time to time.

In [13]:
forced_checkable = {x: checkable_100[x] for x in checkable_100 if x == VOTING_INTENTION}

data = dc.normalise(data, forced_checkable)

For voting-intention; Pattern: Primary -> Selected columns: ['Primary vote L/NP', 'Primary vote ALP', 'Primary vote GRN', 'Primary vote ONP', 'Primary vote UAP', 'Primary vote OTH']
19.63% of rows need normalisation.


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date,Normalisation totals Primary
0,14–17 June 2022,Dynata,Online,1001.0,34.032609,37.326087,13.173913,4.391304,4.391304,7.684783,56.898000,52.102000,2022-06-14,2022-06-15,2022-06-17,101.0
4,31 August – 3 September 2022,Newspoll-YouGov,Online,1505.0,31.000000,37.000000,13.500000,7.000000,2.000000,10.000000,57.000000,43.000000,2022-08-31,2022-09-01,2022-09-03,100.5
10,30 November – 3 December 2022,Newspoll,Online,1508.0,35.000000,39.000000,11.000000,6.000000,1.000000,9.000000,55.000000,45.000000,2022-11-30,2022-12-01,2022-12-03,101.0
11,30 November – 4 December 2022,Resolve Strategic,Online,1611.0,30.000000,42.000000,11.000000,4.000000,2.000000,8.000000,60.000000,40.000000,2022-11-30,2022-12-02,2022-12-04,97.0
14,17–22 January 2023,Resolve Strategic,Online,1606.0,29.000000,42.000000,11.000000,6.000000,2.000000,11.000000,60.000000,40.000000,2023-01-17,2023-01-19,2023-01-22,101.0
18,1–6 February 2023,Essential,Online,1000.0,31.470588,34.617647,17.833333,6.294118,1.049020,15.735294,57.894737,42.105263,2023-02-01,2023-02-03,2023-02-06,107.0
20,15–19 February 2023,Resolve Strategic,Online,1604.0,31.000000,40.000000,10.000000,5.000000,1.000000,11.000000,58.000000,42.000000,2023-02-15,2023-02-17,2023-02-19,98.0
21,15–19 February 2023,Essential,Online,1044.0,32.553191,35.808511,15.191489,6.510638,3.255319,8.680851,55.387097,45.612903,2023-02-15,2023-02-17,2023-02-19,102.0
25,1–5 March 2023,Essential,Online,1141.0,34.357895,34.357895,12.884211,7.515789,2.147368,10.736842,52.688172,47.311828,2023-03-01,2023-03-03,2023-03-05,102.0
26,12–16 March 2023,Resolve Strategic,Online,1600.0,30.000000,39.000000,13.000000,5.000000,1.000000,11.000000,60.000000,40.000000,2023-03-12,2023-03-14,2023-03-16,99.0


For voting-intention; Pattern: 2pp -> Selected columns: ['2pp vote ALP', '2pp vote L/NP']
8.22% of rows need normalisation.


,Date,Brand,Interview mode,Sample size,Primary vote L/NP,Primary vote ALP,Primary vote GRN,Primary vote ONP,Primary vote UAP,Primary vote OTH,2pp vote ALP,2pp vote L/NP,First Date,Mean Date,Last Date,Normalisation totals 2pp
0,14–17 June 2022,Dynata,Online,1001.0,33.695652,36.956522,13.043478,4.347826,4.347826,7.608696,56.898000,52.102000,2022-06-14,2022-06-15,2022-06-17,109.0
21,15–19 February 2023,Essential,Online,1044.0,31.914894,35.106383,14.893617,6.382979,3.191489,8.510638,55.387097,45.612903,2023-02-15,2023-02-17,2023-02-19,101.0
31,12–16 April 2023,Essential,Online,1136.0,31.958763,35.051546,14.432990,6.185567,3.092784,9.278351,54.189474,44.810526,2023-04-12,2023-04-14,2023-04-16,99.0
33,26–30 April 2023,Essential,Online,1130.0,34.042553,35.106383,14.893617,5.319149,2.127660,8.510638,55.819149,43.180851,2023-04-26,2023-04-28,2023-04-30,99.0
34,10–13 May 2023,Resolve Strategic,Online,1610.0,30.303030,42.424242,12.121212,5.050505,2.020202,8.080808,62.220000,39.780000,2023-05-10,2023-05-11,2023-05-13,102.0
41,7–11 June 2023,Essential,Online,1123.0,33.684211,33.684211,16.842105,5.263158,1.052632,9.473684,54.765957,44.234043,2023-06-07,2023-06-09,2023-06-11,99.0
47,19–23 July 2023,Essential,Online,1150.0,34.042553,32.978723,14.893617,7.446809,1.063830,9.574468,53.157895,47.842105,2023-07-19,2023-07-21,2023-07-23,101.0
74,8–12 November 2023,Essential,Online,1150.0,35.789474,33.684211,12.631579,7.368421,2.105263,8.421053,51.552083,49.447917,2023-11-08,2023-11-10,2023-11-12,101.0
79,22–26 November 2023,Essential,Online,1151.0,36.170213,32.978723,13.829787,7.446809,1.063830,8.510638,51.031579,49.968421,2023-11-22,2023-11-24,2023-11-26,101.0
94,24–28 January 2024,Essential,Online,1201.0,35.789474,33.684211,13.684211,7.368421,2.105263,7.368421,50.553191,48.446809,2024-01-24,2024-01-26,2024-01-28,99.0


## Final data validation

Please check any rows identified as a result of this step.

In [14]:
for label, check_list in checkable_100.items():
    row_check = dc.row_sum_check(data[label], check_list, tolerance=1.01)
    if row_check is None or not len(row_check):
        print(f"{label} {check_list} looks good.\n")
        continue
    print(label, check_list)
    display(row_check)
    print("\n")

voting-intention ['Primary', '2pp'] looks good.

attitudinal ["^Dutton (Satisfied|Dissatisfied|Don't Know)", "^Albanese (Satisfied|Dissatisfied|Don't Know)", "Preferred Prime Minister (Dutton|Albanese|Don't Know)"]


,Date,Brand,Interview mode,Sample size,Preferred prime minister Albanese,Preferred prime minister Dutton,Preferred prime minister Don't Know,Preferred prime minister Net,Albanese Satisfied,Albanese Dissatisfied,...,Dutton Satisfied,Dutton Dissatisfied,Dutton Don't Know,Dutton Net,First Date,Mean Date,Last Date,^Dutton (Satisfied|Dissatisfied|Don't Know),^Albanese (Satisfied|Dissatisfied|Don't Know),Preferred Prime Minister (Dutton|Albanese|Don't Know)
15,16–22 November 2022,Morning Consult,Online,NaN,NaN,NaN,NaN,NaN,56.0,31.0,...,NaN,NaN,NaN,NaN,2022-11-16,2022-11-19,2022-11-22,0.0,112.0,0.0
28,29 March – 1 April 2023,Newspoll,Online,1500.0,58.0,26.0,16.0,NaN,56.0,35.0,...,35.0,48.0,21.0,-13.0,2023-03-29,2023-03-30,2023-04-01,104.0,NaN,0.0


## Manage methodology changes

If a pollster firm substantially change the way in which they collect data we need to reflect this in the branding for the poll.

In [15]:
# Essential added education into its weighting
# from the last poll in October 2023.

effective_date = pd.Timestamp("2023-10-24")
change_from = "Essential"
change_to = "Essential 2"
data = dc.methodology(data, effective_date, change_from, change_to)

In [16]:
# Resolve Strategic appears to have changed in 2024

effective_date = pd.Timestamp("2024-01-01")
change_from = "Resolve Strategic"
change_to = "Resolve Strategic 2"
data = dc.methodology(data, effective_date, change_from, change_to)

## Save the checked data

In [17]:
dc.store(data)

## All done

In [18]:
%load_ext watermark
%watermark --python --machine --conda --iversions --watermark

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

conda environment: 312

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.2.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas: 2.2.3

Watermark: 2.5.0



In [19]:
print("Finished")

Finished
